In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {"--complex --clean"}"
)

🔧 FEniCSx Setup Configuration
PETSc type      : complex
Clean install   : True

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Complex PETSc (complex128)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Complex PETSc is installed
   - Use for eigenvalue problems, frequency-domain analysis
   - Some examples may require real PETSc


---

In [2]:
%%fenicsx

"""
DOLFINx Function with Different Data Types
===========================================
Demonstrates creating Functions with different scalar types (dtype)
for real and complex-valued problems.
"""

import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem

# Create a simple unit square mesh
domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.triangle)

# Create a function space (P2 Lagrange elements)
V = fem.functionspace(domain, ("Lagrange", 2))

print(f"Function space dimension: {V.dofmap.index_map.size_global}")
print(f"Number of DOFs per process: {V.dofmap.index_map.size_local}\n")

# 1. Create a real-valued Function (float64)
u_real = fem.Function(V, dtype=np.float64)
print(f"Real Function dtype: {u_real.x.array.dtype}")
print(f"Real Function shape: {u_real.x.array.shape}")

# Set values using interpolation
u_real.interpolate(lambda x: x[0]**2 + x[1]**2)
print(f"Real Function min/max: [{u_real.x.array.min():.4f}, {u_real.x.array.max():.4f}]")

# 2. Create a complex-valued Function (complex128)
u_complex = fem.Function(V, dtype=np.complex128)
print(f"\nComplex Function dtype: {u_complex.x.array.dtype}")
print(f"Complex Function shape: {u_complex.x.array.shape}")

# Set complex values
u_complex.interpolate(lambda x: x[0] + 1j * x[1])
print(f"Complex Function sample values:")
print(f"  First 3 DOFs: {u_complex.x.array[:3]}")

# 3. Direct DOF manipulation
print(f"\n--- Direct DOF Access ---")

# Access DOF vector
dofs_real = u_real.x.array
dofs_complex = u_complex.x.array

print(f"Real DOF vector type: {type(dofs_real)}, dtype: {dofs_real.dtype}")
print(f"Complex DOF vector type: {type(dofs_complex)}, dtype: {dofs_complex.dtype}")

# Modify DOFs directly
u_real.x.array[:] = 5.0  # Set all DOFs to 5.0
u_complex.x.array[:] = 3.0 + 4.0j  # Set all DOFs to 3+4j

print(f"\nAfter modification:")
print(f"  Real Function constant value: {u_real.x.array[0]:.2f}")
print(f"  Complex Function constant value: {u_complex.x.array[0]}")

# 4. Demonstrate use case: wave equation (complex amplitude)
print(f"\n--- Complex Wave Example ---")

# Create a complex wave: e^(i*k*x) where k is wave number
k = 2.0 * np.pi  # Wave number

def complex_wave(x):
    return np.exp(1j * k * x[0])

u_wave = fem.Function(V, dtype=np.complex128)
u_wave.interpolate(complex_wave)

# Extract real and imaginary parts
real_part = np.real(u_wave.x.array)
imag_part = np.imag(u_wave.x.array)

print(f"Complex wave at first 3 DOFs:")
print(f"  Complex: {u_wave.x.array[:3]}")
print(f"  Real part: {real_part[:3]}")
print(f"  Imaginary part: {imag_part[:3]}")
print(f"  Magnitude: {np.abs(u_wave.x.array[:3])}")

# 5. Memory usage comparison
real_memory = u_real.x.array.nbytes
complex_memory = u_complex.x.array.nbytes

print(f"\n--- Memory Usage ---")
print(f"Real Function: {real_memory} bytes")
print(f"Complex Function: {complex_memory} bytes")
print(f"Ratio: {complex_memory / real_memory:.1f}x (complex uses 2x memory)")

print("\n--- Summary ---")
print("✓ Real functions (float64): Standard PDE problems")
print("✓ Complex functions (complex128): Wave equations, quantum mechanics, electromagnetics")
print("✓ Both share same function space but different coefficient storage")

Function space dimension: 289
Number of DOFs per process: 289

Real Function dtype: float64
Real Function shape: (289,)
Real Function min/max: [0.0000, 2.0000]

Complex Function dtype: complex128
Complex Function shape: (289,)
Complex Function sample values:
  First 3 DOFs: [0.875-2.69278509e-18j 1.   -2.69278509e-18j 1.   +1.25000000e-01j]

--- Direct DOF Access ---
Real DOF vector type: <class 'numpy.ndarray'>, dtype: float64
Complex DOF vector type: <class 'numpy.ndarray'>, dtype: complex128

After modification:
  Real Function constant value: 5.00
  Complex Function constant value: (3+4j)

--- Complex Wave Example ---
Complex wave at first 3 DOFs:
  Complex: [0.70710678-7.07106781e-01j 1.        -2.44929360e-16j
 1.        -2.44929360e-16j]
  Real part: [0.70710678 1.         1.        ]
  Imaginary part: [-7.07106781e-01 -2.44929360e-16 -2.44929360e-16]
  Magnitude: [1. 1. 1.]

--- Memory Usage ---
Real Function: 2312 bytes
Complex Function: 4624 bytes
Ratio: 2.0x (complex uses 2x